In [6]:
import tools.processing as pre
import tools.architectures as nn

In [7]:
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

In [9]:
text[:100]

"as real as it seems the american dream\nain't nothing but another calculated schemes\nto get us locked"

In [4]:
alphabet = pre.Alphabet(text)

HIDDEN_LAYER_SIZE = 128
VOCAB_SIZE = alphabet.get_size()
TIMESTEPS = 50

EPOCHS = 20
BATCH_SIZE = 128


data, labels = alphabet.making_one_hot(text, TIMESTEPS)


rnn = nn.SingleLayerRNN(name = "basic")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

rnn.train(data, labels, alphabet, epochs=EPOCHS, batch_size=BATCH_SIZE)



Epoch 1/20
Loss:    	 3.43237042427063
Accuracy:	 1.1122249364852905
Seed:as real as it seems the american dream
ain't nothi
Result:as real as it seems the american dream
ain't nothio toot y ineee ubu  h   lee o e  l   tir  eteie y u w y    e s    ui  l  uoh  te
 e t ey  euit w eo e   ae   ueenoe si weu eo re e   l  obrt t  h  a  o pa  lt   r  s y  ei l a  a  ono t  s  u i e'    y    o     s  e el   t lo nteye e ln e ieins  b    r     r  na ta nrcinat  eyet e   r  o y      nte  dc   s d  yo  bo aa s   ti ad e rt      to r eo   ya eu  t   a  yd  eh ie  t iae    e ott   ey  s e ooe  nun  u e r  e  tn ht   otehm   uioe  oar t  e   a it  t e   esot es u  eoe  yea
  e  t    uir 


Epoch 2/20
Loss:    	 2.9421982765197754
Accuracy:	 17.848106384277344
Seed:as real as it seems the american dream
ain't nothi
Result:as real as it seems the american dream
ain't nothi  ohi  aid e ne  hea oy a e  to e  e e e thn aii u o o o mo uyt l
ta a e i yoty i ae e  o a or mu oo   mo  rae t eo  oui h ia  oo

KeyboardInterrupt: 